In [149]:
import numpy as np

signals_per_digit = {'0': 6, \
                     '1': 2, \
                     '2': 5, \
                     '3': 5, \
                     '4': 4, \
                     '5': 5, \
                     '6': 6, \
                     '7': 3, \
                     '8': 7, \
                     '9': 6}

class DAY_08:
    def __init__(self, fname, number=10):
        self.__fname = fname
        self.__number = number
        self.__sum_over_all = 0
        
    def read_signals(self):
        self.__digits = []
        self.__digits_flatten = []
        self.__signals_flatten = []
        self.__signal_sizes = []
        with open(self.__fname, "r") as f:
            for line in f.readlines():
                self.__digits.append([line.split(" ")[-i].rstrip() for i in range(1, 5)])
                for digit in line.split(" ")[-4:]:
                    self.__digits_flatten.append("".join(sorted(digit.rstrip())))
                for signal in line.split(" ")[:10]:
                    self.__signals_flatten.append("".join(sorted(signal.rstrip())))
                    self.__signal_sizes.append(len(signal.rstrip()))
                    
    def show_digits(self):
        print(self.__digits_flatten)
        
    def show_input(self):
        print(self.__signals_flatten, self.__digits_flatten)
        
    def show_signal_sizes(self):
        for i in range(0, len(self.__signal_sizes)//self.__number):
            print(self.__signal_sizes[self.__number*i:self.__number*(i+1)])
            print(self.__signals_flatten[self.__number*i:self.__number*(i+1)])
            
    def show_sum_over_all(self):
        print(self.__sum_over_all)
            
    #Aufgabe 1
    def count_sizes(self):
        out = [0,0,0,0]
        for digit in self.__digits_flatten:
            if len(digit) == signals_per_digit['1']: out[0] += 1
            if len(digit) == signals_per_digit['4']: out[1] += 1
            if len(digit) == signals_per_digit['7']: out[2] += 1
            if len(digit) == signals_per_digit['8']: out[3] += 1
        print(out, sum(out))
        
    #Aufgabe 2
    def find_wires(self, verbose=False):
        #Nicht auf effizienz getrimmt ...
        for i in range(0, len(self.__signal_sizes)//self.__number):
            
            #Suche die 10 zusammengehörigen Signale
            wires = ["" for j in range(10)]
            sizes = np.array(self.__signal_sizes[self.__number*i:self.__number*(i+1)])
            signals = np.array(self.__signals_flatten[self.__number*i:self.__number*(i+1)])
            digits = np.array(self.__digits_flatten[4*i:4*(i+1)])
            
            #Schreibe die eindeutigen Zahlen raus
            for j in [1,4,7,8]:
                wires[j] = signals[sizes == signals_per_digit[str(j)]][0]
                
            #Suche die 6: Wenn ein String nicht die Zeichen der 1 enthält und 6 Zeichen enthält, ist es die 6
            for sechs in signals[sizes == signals_per_digit['6']]:
                if wires[1][0] in sechs and wires[1][1] not in sechs:
                    wires[6] = sechs
                elif wires[1][0] not in sechs and wires[1][1] in sechs:
                    wires[6] = sechs
                    
            #Suche die 5: Die 5 enthält einen Buchstaben weniger als die 6. Die verbleibenden 
            #Buchstaben sind identisch.
            fuenf = [True, True, True]
            fuenfen = signals[sizes == signals_per_digit['5']]
            for index in range(fuenfen.size):
                for letter in fuenfen[index]:
                    if letter not in wires[6]:
                        fuenf[index] = False
                        break
            wires[5] = fuenfen[fuenf][0]

            #Suche die 2: Die 2 ist das letzte Element, welches nicht beide Buchstaben der 1 enthält
            for index in signals:
                if index not in wires and ((wires[1][0] in index and wires[1][1] not in index) or \
                                       (wires[1][0] not in index and wires[1][1] in index)):
                    wires[2] = index
                    break
            #Suche die 3: Die 3 hat fünf Elemente und enthält beide Buchstaben der 1
            for index in signals:
                if len(index) == signals_per_digit['3'] and wires[1][0] in index and wires[1][1] in index:
                    wires[3] = index
                    break
            
            #Suche die 9: Die 9 enthält alle Buchstaben der 4 und hat die Länge 6
            for index in signals:
                if len(index) == signals_per_digit['9'] and wires[4][0] in index and \
                                                            wires[4][1] in index and \
                                                            wires[4][2] in index and \
                                                            wires[4][3] in index:
                    wires[9] = index
            
            #Suche die 0: Die 0 ist die letzte verbleibende Zahl
            for index in signals:
                if index not in wires:
                    wires[0] = index
                    break
                    
            #Decodiere den Output der U-Boot-Anzeige
            digits_decoded = []
            for output in digits:
                for combination in range(len(wires)):
                    if wires[combination] == output:
                        digits_decoded.append(combination)
            if verbose:
                print(wires, digits, digits_decoded)
            self.__sum_over_all += np.sum(np.array(digits_decoded)*10**np.array([3,2,1,0]))

                
            
day08_test_1 = DAY_08("example.txt")
day08_test_1.read_signals()
day08_test_1.show_digits()
day08_test_1.count_sizes()

day08_test_1 = DAY_08("input.txt")
day08_test_1.read_signals()
day08_test_1.count_sizes()

['abcdefg', 'bcdef', 'bcdefg', 'bceg', 'bcdefg', 'bcg', 'abcdefg', 'cg', 'cg', 'cg', 'abcdfg', 'bcg', 'abcdef', 'abcde', 'acdefg', 'bc', 'cefg', 'abcdefg', 'bfg', 'abefg', 'abcdefg', 'abce', 'ac', 'abcdefg', 'cefg', 'bcdef', 'cefg', 'abcdefg', 'de', 'abcefg', 'abcdg', 'bcefg', 'abcdefg', 'bcg', 'cg', 'bcg', 'aefg', 'abcfg', 'fg', 'abceg']
[8, 6, 5, 7] 26
[68, 61, 51, 67] 247


In [150]:
day08_test_2 = DAY_08("example.txt")
day08_test_2.read_signals()
day08_test_2.find_wires(verbose=True)
day08_test_2.show_sum_over_all()

day08_puzzle_2 = DAY_08("input.txt")
day08_puzzle_2.read_signals()
day08_puzzle_2.find_wires()
day08_puzzle_2.show_sum_over_all()

['abdefg', 'be', 'abcdf', 'bcdef', 'bceg', 'cdefg', 'acdefg', 'bde', 'abcdefg', 'bcdefg'] ['abcdefg' 'bcdef' 'bcdefg' 'bceg'] [8, 3, 9, 4]
['abcdfg', 'cg', 'abcde', 'bcdeg', 'cefg', 'bdefg', 'abdefg', 'bcg', 'abcdefg', 'bcdefg'] ['bcdefg' 'bcg' 'abcdefg' 'cg'] [9, 7, 8, 1]
['bcdefg', 'cg', 'abcde', 'abcdg', 'acfg', 'abdfg', 'abdefg', 'bcg', 'abcdefg', 'abcdfg'] ['cg' 'cg' 'abcdfg' 'bcg'] [1, 1, 9, 7]
['bcdefg', 'bc', 'abdeg', 'abcde', 'abcf', 'acdef', 'acdefg', 'bcd', 'abcdefg', 'abcdef'] ['abcdef' 'abcde' 'acdefg' 'bc'] [9, 3, 6, 1]
['abcdfg', 'fg', 'abdef', 'abefg', 'cefg', 'abceg', 'abcdeg', 'bfg', 'abcdefg', 'abcefg'] ['cefg' 'abcdefg' 'bfg' 'abefg'] [4, 8, 7, 3]
['acdefg', 'ac', 'bcdfg', 'abcfg', 'abce', 'abefg', 'abdefg', 'acf', 'abcdefg', 'abcefg'] ['abcdefg' 'abce' 'ac' 'abcdefg'] [8, 4, 1, 8]
['abdefg', 'fg', 'abcdg', 'bcdfg', 'cefg', 'bcdef', 'abcdef', 'dfg', 'abcdefg', 'bcdefg'] ['cefg' 'bcdef' 'cefg' 'abcdefg'] [4, 5, 4, 8]
['acdefg', 'de', 'abcdg', 'bcdeg', 'bdef', 'bcefg'